In [ ]:
## This is our fourth and final attempt at making profit from the stock market. In this attempt we were able to successfully create the pipline to compute and filter the desired stocks, as well as creating a pipeline to take advantage of quantopians risk-loadings computations. These are useful in refining our portfolio by using constraints, which in turn help optimize our buys and sells. We make the computations, set our perameters, and minimize risk, all before passing all of that information to the optimizer that finalizes our portfolio. Although this final attempt was our most successful, it was not very profitable and did not make more than the SPY did. This is due to the oversaturation of this well-known, basic strategy in quant finance. With many people utilizing this strategy, the trends observed are quickly adjusted by the market. This is a challenge we faced and ultimately could not overcome, despite learning a ton throughout the process.

import quantopian.algorithm as algo
import quantopian.optimize as opt
from quantopian.pipeline import Pipeline
from quantopian.algorithm import attach_pipeline, pipeline_output
from quantopian.pipeline.factors import SimpleMovingAverage
from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.experimental import risk_loading_pipeline
from quantopian.pipeline.data.psychsignal import stocktwits
from quantopian.pipeline.data import Fundamentals

# Global constraint parameters to manage the total long and short positions our portfolio will
# maintain, and the gross leverage allowed at any given time. Leverage is essentially boroughed
# money that will increase profits, even when the percentage increase is small. So by leveraging,
# we can profit greatly, although there is more risk, and thus a max gross leverage is needed.
MAX_GROSS_LEVERAGE = 1.0
NUM_LONG_POSITIONS = 300
NUM_SHORT_POSITIONS = 300

#initialized once
def initialize(context):
    # Set the commission and slippage models to be implemented in backtesting. (See attempt three
    # comments for a detailed understanding of commission and slippage and why they need to be
    # accounted for)
    set_commission(commission.PerShare(cost=0.0, min_trade_cost=0))
    set_slippage(slippage.VolumeShareSlippage(volume_limit=1, price_impact=0))
    
    # Create our stock selecting pipeline
    algo.attach_pipeline(make_pipeline(), 'comp_pipe')
    
    # Create our risk model factor pipeline to be implemented in optimization
    algo.attach_pipeline(risk_loading_pipeline(), 'risk_pipe')
    
    # Schedule our rebalance function
    algo.schedule_function(func=rebalance,
                           date_rule=algo.date_rules.every_day(),
                           time_rule=algo.time_rules.market_open(hours=0, minutes=30),
                           half_days=True)

    # Record our portfolio variables at the end of day
    algo.schedule_function(func=record_vars,
                           date_rule=algo.date_rules.every_day(),
                           time_rule=algo.time_rules.market_close(),
                           half_days=True)

def make_pipeline():
    
    # The screen will only let chosen stocks through if they belong to the universe. (See attempt
    # three for a more detailed understanding of universe)
    universe = QTradableStocksUS()
    
    # Using fundamentals data 'earnings before interest and taxes' (ebit) and 'enterprise value'
    # to compute a standardized value variable.
    # Return on equity (roe) is other fundamentals data that is used to compute a quality variable
    # A simple moving average of the sentiment score for each stock going back 3 days is also used.
    value = Fundamentals.ebit.latest / Fundamentals.enterprise_value.latest
    quality = Fundamentals.roe.latest
    sentiment_score = SimpleMovingAverage(
        inputs=[stocktwits.bull_minus_bear],
        window_length=3,
    )
    
    #Compute the simple moving average over 50 days
    sma_50 = SimpleMovingAverage(
        inputs = [USEquityPricing.close],
        window_length = 50,
    )
    
    # Compute the simple moving average over 20 days
    sma_20 = SimpleMovingAverage(
        inputs = [USEquityPricing.close],
        window_length = 20,
    )
    
    # Winsorize factor values. This is a computation provided by quantopian that will lessen the
    # impact of outliers.
    value_winsorized = value.winsorize(min_percentile=0.05, max_percentile=0.95)
    quality_winsorized = quality.winsorize(min_percentile=0.05, max_percentile=0.95)
    sentiment_score_winsorized = sentiment_score.winsorize(min_percentile=0.05,                                                                             max_percentile=0.95)

    # Combine winsorized factors, z-scoring them to equalize their influence (also provided).
    combined_factor = (
        value_winsorized.zscore() + 
        quality_winsorized.zscore() + 
        sentiment_score_winsorized.zscore() 
    )
    
    # Build filters representing the top and bottom baskets of stocks by our
    # combined ranking system. Mask tells the pipeline to use these as our 
    # tradeable universe each day.
    longs = combined_factor.top(NUM_LONG_POSITIONS, mask=universe)
    shorts = combined_factor.bottom(NUM_SHORT_POSITIONS, mask=universe)

    # The final output of our pipeline should only include
    # the top/bottom 300 stocks by our criteria
    long_short_screen = (longs | shorts)

    # Create pipeline using the predefined factors and screen
    pipe = Pipeline(
        columns = {
            'longs': longs,
            'shorts': shorts,
            'sma_20': sma_20,
            'sma_50': sma_50,
            'combined_factor': combined_factor
        },
        screen = long_short_screen
    )
    
    return pipe

def before_trading_start(context, data):

    # Get our main computations pipeline output data
    context.output_data = algo.pipeline_output('comp_pipe')
    
    # Get our risk loadings output
    context.output_risk = algo.pipeline_output('risk_pipe')
              
def record_vars(context, data):
    # Plot the number of positions over time.
    algo.record(num_positions=len(context.portfolio.positions))

def rebalance(context, data):
    # Get the output data from the pipelines to be used within this method
    output_data = context.output_data
    output_risk = context.output_risk
              
    # Define an objective to be used by the optimize method
    # This will develop our portfolio based on maximizing the alpha of our algorithm
    # Alpha is the 'edge', so to speak, that our algorithm has on the market. So rather than
    # maximizing the return, which the market also provides (it's always going up a little), we 
    # can maximize the algorithm to give us the most 'advantage' or 'edge' our algo can have
    # when compared to the market. (See blog post for further detail)
    objective = opt.MaximizeAlpha(output_data.combined_factor)
              
    # Define the list of constraints and constrain our maximum gross leverage
    # (Leverage is defined at the top of this document and in the blog post)
    constraints = []
    constraints.append(opt.MaxGrossExposure(MAX_GROSS_LEVERAGE))

    # Optimize our algorithm to remain dollar neutral, investing equal parts in long and short
    # trades. This is will hedge our portfolio against rapid upward or downward movements by
    # ensuring we will lose as much as we make in those worse-case scenarios.
    constraints.append(opt.DollarNeutral())

    # Add the RiskModelExposure constraint to use the risk model provided by Quantopian.
    # This will use the risk loadings provided by our risk pipeline, an assessment that quantopian
    # computes for a portfolio, to be added to our constraints.
    neutralize_risk_factors = opt.experimental.RiskModelExposure(
        risk_model_loadings=output_risk,
        version=0
    )
    constraints.append(neutralize_risk_factors)

    # Constrain our portfolio to invest equally in stocks that fit our long and short
    # computations ensuring that we don't invest in only one or few stocks.
    constraints.append(
        opt.PositionConcentration.with_equal_bounds(
            min=-NUM_SHORT_POSITIONS,
            max=NUM_LONG_POSITIONS
        ))

    # Order optimal portfolio to maximize alpha, minimize risk, and meet our constraints
    algo.order_optimal_portfolio(
        objective=objective,
        constraints=constraints
    )